<a href="https://colab.research.google.com/github/seongheek/econtheory/blob/main/11%EC%A3%BC%EC%B0%A8_%EA%B0%95%EC%9D%98_%EC%9D%B4%EC%82%B0%EB%AA%A8%ED%98%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **이항반응모형 분석**
LPM, 로짓, 프로빗을 분석해보자.


경제이해력 데이터를 사용하여 분석해보자.

In [1]:
#데이터 불러오고 전처리
import pandas as pd
import numpy as np

df=pd.read_excel('econliteracy.xlsx', index_col=0)

df = df.rename(columns={'sq1': 'gender', 'sq2': 'age', 'sq3': 'region', 'sq4': 'job', 'sq5': 'edu', 'sq6': 'income'})  #열 이름 바꾸기
df = df.rename(columns={'A4_A': 'channel', 'A5_A': 'econedu'})  #열 이름 바꾸기
df=df.dropna(subset=['gender'])        #성별 정보가 누락된 샘플 제거

# 변수정리

df['gender'] = df['gender'].replace(2, 0)
df['income'] = df['income'].replace(8, 7)
df['job'] = df['job'].replace(6, 5)
df['edu'] = df['edu'].replace(1, 2)
df['region2'] = df['region'].apply(
    lambda x: 1 if x in [1, 4, 9]
    else 2 if x in [6, 7, 8, 10, 11, 12]
    else 3 if x in [2, 3, 7, 15, 16]
    else 4 if x in [5, 13, 14]
    else None
)
#보기 6~8 4로 통합
df['channel'] = df['channel'].replace(7, 4)
df['channel'] = df['channel'].replace(8, 4)
df['channel'] = df['channel'].replace(6, 4)
df['channel'].value_counts().sort_index()

#소득 카테고리 줄이기
df['income'] = df['income'].replace(7, 6)
df['income'] = df['income'].replace(1, 2)

#직업 카테고리 줄이기
df['job'] = df['job'].replace(4, 3)
df['job'] = df['job'].replace(6, 5)
df['job'] = df['job'].replace(8, 7)
df['job'] = df['job'].replace(9, 7)

#학교밖 경제교육경험 더미
df['econedu'] = df['econedu'].replace(2, 0)


In [2]:
#목돈투자시 예금 여부 더미를 생성해보자.
df['save'] = df['A9_A'].apply(lambda x: 1 if x in [1] else 0)
df['save'].describe()

,save
count,297.000000
mean,0.434343
std,0.496507
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


# LPM 모형

In [3]:
import statsmodels.formula.api as smf
model = smf.ols('save ~ age + I(age**2) + gender+ edu + C(channel)', data=df).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                   save   R-squared:                       0.101
Model:                            OLS   Adj. R-squared:                  0.076
Method:                 Least Squares   F-statistic:                     4.031
Date:                Wed, 14 May 2025   Prob (F-statistic):           0.000147
Time:                        08:19:55   Log-Likelihood:                -197.22
No. Observations:                 297   AIC:                             412.4
Df Residuals:                     288   BIC:                             445.7
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           1.1737      0.353     

# 로짓 모형

In [4]:
import statsmodels.formula.api as smf
model = smf.logit('save ~ age + I(age**2) + gender+ edu + C(channel)', data=df).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.631936
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                   save   No. Observations:                  297
Model:                          Logit   Df Residuals:                      288
Method:                           MLE   Df Model:                            8
Date:                Wed, 14 May 2025   Pseudo R-squ.:                 0.07679
Time:                        08:19:57   Log-Likelihood:                -187.68
converged:                       True   LL-Null:                       -203.30
Covariance Type:            nonrobust   LLR p-value:                 0.0001282
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           3.0660      1.585      1.935      0.053      -0.040       6.172
C(channel)[T.2

In [5]:
# 계수 추출
coefficients = model.params

# 오즈비 (odds ratio)
odds_ratios = np.exp(coefficients)

print(odds_ratios)

Intercept          21.454983
C(channel)[T.2]     0.184115
C(channel)[T.3]     0.682971
C(channel)[T.4]     0.447234
C(channel)[T.5]     1.326148
age                 0.863197
I(age ** 2)         1.001731
gender              0.947849
edu                 0.854768
dtype: float64


# 프로빗 모형

In [6]:
import statsmodels.formula.api as smf
model = smf.probit('save ~ age + I(age**2) + gender+ edu + C(channel)', data=df).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.632102
         Iterations 5
                          Probit Regression Results                           
Dep. Variable:                   save   No. Observations:                  297
Model:                         Probit   Df Residuals:                      288
Method:                           MLE   Df Model:                            8
Date:                Wed, 14 May 2025   Pseudo R-squ.:                 0.07655
Time:                        08:20:01   Log-Likelihood:                -187.73
converged:                       True   LL-Null:                       -203.30
Covariance Type:            nonrobust   LLR p-value:                 0.0001335
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           1.9103      0.975      1.960      0.050       0.000       3.821
C(channel)[T.2

In [7]:
#한계효과
margeff = model.get_margeff()
margeff.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
       Probit Marginal Effects       
=====================================
Dep. Variable:                   save
Method:                          dydx
At:                           overall
===================================================================================
                     dy/dx    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
C(channel)[T.2]    -0.3611      0.164     -2.207      0.027      -0.682      -0.040
C(channel)[T.3]    -0.0863      0.068     -1.275      0.202      -0.219       0.046
C(channel)[T.4]    -0.1754      0.097     -1.810      0.070      -0.365       0.015
C(channel)[T.5]     0.0601      0.190      0.316      0.752      -0.313       0.433
age                -0.0327      0.012     -2.835      0.005      -0.055      -0.010
I(age ** 2)         0.0004      0.000      3.232      0.001       0.000       0.001
gender             -0.0129      0.055     -0.234      0.815      -0.121       0.095
edu                -0.0374      0.071     -0.524      0.600      -0.177       0.102
===================================================================================
"""

한계효과는 다른 통제변수들의 평균에서의 한계효과(marginal effect at the mean)를 의미한다.